In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [2]:
data = pd.read_pickle('./Data/blue_niles_df.pkl')
data.head()

,Shape,Price,Discount Price,Carat,Cut,Color,Clarity,Polish,Symmetry,Fluorescence,Depth,Table,L/W,Price/Ct,Culet,Delivery Date,Last Available Date,First Available Date
Stock No.,,,,,,,,,,,,,,,,,,
LD12511939,Round,636.0,560.0,0.50,Ideal,J,SI2,Excellent,Very Good,Medium,61.8,57.0,1.01,1120.0,None,2020-05-06,2020-04-04,2020-04-03
LD13142833,Round,629.0,563.0,0.50,Ideal,J,SI2,Excellent,Very Good,Faint,60.9,59.0,1.01,1126.0,None,2020-05-12,2020-04-09,2020-04-03
LD13485098,Round,578.0,578.0,0.50,Very Good,K,SI2,Excellent,Excellent,None,64.5,58.0,1.01,1156.0,None,2020-05-28,2020-04-14,2020-04-03
LD12511933,Round,581.0,581.0,0.51,Ideal,J,SI2,Excellent,Very Good,Faint,62.0,58.0,1.01,1139.0,None,2020-06-01,2020-04-16,2020-04-03
LD13258334,Round,569.0,569.0,0.50,Very Good,K,SI2,Excellent,Excellent,Faint,64.3,55.0,1.00,1138.0,None,2020-06-01,2020-04-16,2020-04-03


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87644 entries, LD12511939 to LD13677110
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Shape                 87644 non-null  object 
 1   Price                 87644 non-null  float64
 2   Discount Price        87644 non-null  float64
 3   Carat                 87644 non-null  float64
 4   Cut                   87644 non-null  object 
 5   Color                 87644 non-null  object 
 6   Clarity               87644 non-null  object 
 7   Polish                87644 non-null  object 
 8   Symmetry              87644 non-null  object 
 9   Fluorescence          87644 non-null  object 
 10  Depth                 87644 non-null  float64
 11  Table                 87644 non-null  float64
 12  L/W                   87644 non-null  float64
 13  Price/Ct              87644 non-null  float64
 14  Culet                 87644 non-null  object 
 15  Delivery D

In [4]:
cat_col = ['Shape', 'Cut', 'Color', 'Clarity', 'Polish', 'Symmetry', 'Fluorescence', 'Culet']
num_col = ['Price', 'Discount Price', 'Carat', 'Depth', 'Table', 'L/W', 'Price/Ct']
date_col = ['Delivery Date', 'Last Available Date', 'First Available Date']

In [5]:
for col in cat_col:
    print(data[col].unique())

['Round']
['Ideal' 'Very Good' 'Good' 'Astor']
['J' 'K' 'I' 'H' 'G' 'F' 'E' 'D']
['SI2' 'VS2' 'SI1' 'VS1' 'VVS1' 'VVS2' 'IF' 'FL']
['Excellent' 'Very Good' 'Good']
['Very Good' 'Excellent' 'Good']
['Medium' 'Faint' 'None' 'Strong' 'Strong Blue' 'Medium Blue'
 'Very Strong' 'Medium Yellow' 'Faint Blue' 'Very Strong Blue'
 'Strong Yellow' 'Medium White']
['None' 'Very Small' 'Small' 'Medium' 'Pointed']


In [6]:
data.Cut.value_counts()

Ideal        65702
Very Good    16597
Good          4681
Astor          664
Name: Cut, dtype: int64

# Transformer Test

In [7]:
from preprocessing.transformer import ColumnSelector, DateSplitTransformer, DateDeltaTransformer
from preprocessing.imputer import DateImputer

In [30]:
column_selector = ColumnSelector(col_name=num_col)
date_split_transformer = DateSplitTransformer(date_type='Delivery Date')
date_diff_transformer = DateDeltaTransformer(delta_type='deliver_days')
date_imputer = DateImputer(first_available_date='earliest', last_available_date='latest', deliver_date='latest')

In [27]:
column_selector.fit_transform(data).head()

,Price,Discount Price,Carat,Depth,Table,L/W,Price/Ct
Stock No.,,,,,,,
LD12511939,636.0,560.0,0.50,61.8,57.0,1.01,1120.0
LD13142833,629.0,563.0,0.50,60.9,59.0,1.01,1126.0
LD13485098,578.0,578.0,0.50,64.5,58.0,1.01,1156.0
LD12511933,581.0,581.0,0.51,62.0,58.0,1.01,1139.0
LD13258334,569.0,569.0,0.50,64.3,55.0,1.00,1138.0


In [31]:
date_split_transformer.fit_transform(data)

array([[2020,    5,    6],
       [2020,    5,   12],
       [2020,    5,   28],
       ...,
       [2020,    4,   20],
       [2020,    4,   20],
       [2020,    6,    1]], dtype=int64)

In [32]:
date_diff_transformer.fit_transform(data)

array([[32],
       [33],
       [44],
       ...,
       [ 4],
       [ 4],
       [46]], dtype=int64)

In [33]:
fake_date_data = data.head(10)
fake_date_data.loc[['LD12511939', 'LD12511933', 'LD13258334', 'LD13258333'], 'Delivery Date'] = None
fake_date_data.loc[['LD13142833', 'LD12511933', 'LD12146902', 'LD13258333'], 'Last Available Date'] = None
fake_date_data.loc[['LD13485098', 'LD13258334', 'LD12146902', 'LD13258333'], 'First Available Date'] = None
date_imputer.fit_transform(fake_date_data).head()

,Shape,Price,Discount Price,Carat,Cut,Color,Clarity,Polish,Symmetry,Fluorescence,Depth,Table,L/W,Price/Ct,Culet,Delivery Date,Last Available Date,First Available Date
Stock No.,,,,,,,,,,,,,,,,,,
LD12511939,Round,636.0,560.0,0.50,Ideal,J,SI2,Excellent,Very Good,Medium,61.8,57.0,1.01,1120.0,None,2020-06-01,2020-04-04,2020-04-03
LD13142833,Round,629.0,563.0,0.50,Ideal,J,SI2,Excellent,Very Good,Faint,60.9,59.0,1.01,1126.0,None,2020-05-12,2020-04-16,2020-04-03
LD13485098,Round,578.0,578.0,0.50,Very Good,K,SI2,Excellent,Excellent,None,64.5,58.0,1.01,1156.0,None,2020-05-28,2020-04-14,2020-04-03
LD12511933,Round,581.0,581.0,0.51,Ideal,J,SI2,Excellent,Very Good,Faint,62.0,58.0,1.01,1139.0,None,2020-06-01,2020-04-16,2020-04-03
LD13258334,Round,569.0,569.0,0.50,Very Good,K,SI2,Excellent,Excellent,Faint,64.3,55.0,1.00,1138.0,None,2020-06-01,2020-04-16,2020-04-03


# Model Test

In [9]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from model.pricer import DiamondPricer

In [21]:
diamond_pricer = DiamondPricer()
diamond_pricer.build_preprocessor()
diamond_pricer.get_params()

{'algo_params': None,
 'cv_params': None,
 'preprocessor_params': {'target_col': ['Price'],
  'cat_col': ['Shape',
   'Cut',
   'Color',
   'Clarity',
   'Polish',
   'Symmetry',
   'Fluorescence',
   'Culet'],
  'cat_encoder': OrdinalEncoder(categories='auto', dtype=<class 'numpy.float64'>),
  'cat_imputer': SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                missing_values=nan, strategy='most_frequent', verbose=0),
  'num_col': ['Carat', 'Depth', 'Table', 'L/W'],
  'num_imputer': SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                missing_values=nan, strategy='median', verbose=0),
  'num_scaler': StandardScaler(copy=True, with_mean=True, with_std=True),
  'date_col': ['First Available Date', 'Last Available Date', 'Delivery Date'],
  'date_imputer': DateImputer(deliver_date=None, first_available_date=None,
              last_available_date=None),
  'date_splitter': ['First Available Date'],
  'date_delta': ['deliver_days', 'in_sto

In [19]:
diamond_pricer.preprocessor.fit_transform(data)

array([[  0.,   2.,   6., ...,   3.,  32.,  -1.],
       [  0.,   2.,   6., ...,   3.,  33.,  -6.],
       [  0.,   3.,   7., ...,   3.,  44., -11.],
       ...,
       [  0.,   2.,   7., ...,  16.,   4.,   0.],
       [  0.,   2.,   3., ...,  16.,   4.,   0.],
       [  0.,   2.,   3., ...,  16.,  46.,   0.]])